In [158]:
#import numpy as np
import autograd.numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autograd import grad
from autograd import elementwise_grad as egrad
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [138]:
#loading data
filname='Real estate valuation data set.xlsx'
df=pd.read_excel(filname)
data=np.array(df)

train_X,test_X,train_Y, test_Y = train_test_split(data[:,1:7],data[:,-1])
train_Y=np.array([train_Y]).T
test_Y=np.array([test_Y]).T

In [139]:
for i in [train_X,test_X,train_Y, test_Y]:      #verifying  the shapes compatibility
    print (i.shape)

(310, 6)
(104, 6)
(310, 1)
(104, 1)


In [159]:
def normalize(X):
    X_=[]
    for i in range(len(X[0])):
        X_.append(X[:,i]/X[:,i].max())
    X_=np.array(X_).T
    return X_
def RMSE(y1,y2):
    #calculating RMS
    y1=np.transpose(y1)[0]
    y2=np.transpose(y2)[0]
    rmse=(sum(((y1-y2)**2))/len(y1))**0.5
    return rmse
def predict(X,theta):
    X=np.array(X)
    theta=np.array(theta)
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    return np.matmul(X,theta) 

def normalEquationRidgeRegression(X, y, lmda=0.01):
    X=np.array(X)
    y=np.array(y)
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    X_t=np.transpose(X)
    A_=np.matmul(X_t,X)            #=X_t*X
    I_=np.diag(np.ones(len(A_)))
    I_[0][0]=0                     #=I_
    A=A_+lmda*I_                   
    theta=0
    if np.linalg.det(A)==0:
        theta=normalEquationRegression(X[:,1:], y)
    else:
        A=np.linalg.inv(A)
        B=np.matmul(X_t,y)
        theta=np.matmul(A,B)
    return theta



def P_j(X,y, j, theta):   #function to be used in coordinate descent and lasso
    tmp_X=np.array([i for i in X])
    tmp_theta=np.array([i for i in theta])
    np.delete(tmp_theta,j,axis=0)
    np.delete(tmp_X,j,axis=1)
    y_=np.matmul(tmp_X,tmp_theta)
    Pj=0
    for index,x in enumerate(tmp_X):
        Pj+=x[j]*(y[index][0]-y_[index][0])
    return Pj
def Z_j(X,j):
    X_tmp=X[:,j]
    return sum(X_tmp**2)
def coodrdinateDescentRegression(X, y, epoch=500):
    X=np.array(X)
    y=np.array(y)
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    theta=np.random.randn(len(X[0]),1)  #random initialization of parameters
    for _ in range(epoch):
        theta_tmp=np.array([i for i in theta])
        for j in range(len(theta)):
            Pj=P_j(X,y,j,theta)
            Zj=Z_j(X,j)
            theta_tmp[j][0]=Pj/Zj
        theta=theta_tmp
    return theta


def coodrdinateDescentLasso(X, y, lmda=0.1, epoch=500):
    X=np.array(X)
    y=np.array(y)
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    theta=np.random.randn(len(X[0]),1)
    for _ in range(epoch):
        theta_tmp=np.array([i for i in theta])
        for j in range(len(X[0])):
            Pj=P_j(X,y,j,theta)
            if Pj<-lmda/2:
                theta_tmp[j][0]=(Pj+lmda/2)/(2*(j+1))
            elif Pj>lmda/2:
                theta_tmp[j][0]=(Pj-lmda/2)/(2*(j+1))
            else:
                theta_tmp[j][0]=0
        theta=theta_tmp
    return theta


def grad_eval(x_i, y_i , theta):  #x_i is 1x(d+1) matrix and y_i is a 1x1 vector, it returns the d(error)/d(theta) 
    theta_new= 2*np.matmul(np.matmul(np.c_[x_i],np.c_[x_i].T),theta)- 2*np.dot(np.c_[x_i], np.array([y_i]))
    return theta_new
def sgdRegression(X, y, alpha = 0.1, epoch=500):
    X=np.array(X)
    y=np.array(y)
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    theta=np.random.randn(len(X[0]),1)  #random initialization of parameters
    
    for i in range(epoch):
        data=np.insert(X,len(X[0]),values=y.T[0],axis=1)
        np.random.shuffle(data)
        X_new=data[:,:-1]
        y_new=data[:,-1:]
        for index,x_i in enumerate(X_new):
            y_i=y_new[index]
            #print(y_i)
            theta=theta-alpha*grad_eval(x_i,y_i,theta)
    
    return theta
    
def Error(theta):     #requires globbal declaration of X,y,lmda
    global train_X
    global train_Y
    global lmda
    y=train_Y
    y_=np.matmul(np.insert(train_X,0,values=np.ones(len(train_X)),axis=1),theta)
    return (y-y_)**2+lmda*(sum(abs(theta))[0])

def gradientDescentAutogradLasso(X, y, alpha = 0.1, lmda=0.01, epoch=500):
    X=np.insert(X,0,values=np.ones(len(X)),axis=1)
    theta=np.random.randn(len(X[0]),1)
    derivative=egrad(Error)
    for i in range(epoch):
        #y_=np.matmul(X,theta)
        theta=theta-alpha*np.array(derivative(theta))
    return theta


In [123]:
theta=normalEquationRidgeRegression(train_X, train_Y, lmda=0.01)
theta
[RMSE(train_Y, predict(train_X,theta)), RMSE(test_Y, predict(test_X,theta))]

[9.214756118918338, 7.739431723967597]

In [124]:
theta=coodrdinateDescentLasso(normalize(train_X), train_Y, lmda=1, epoch=10)
[RMSE(train_Y, predict(normalize(train_X),theta)), RMSE(test_Y, predict(normalize(test_X),theta))]

[1.7764322053406734e+26, 1.7814256915499223e+26]

In [134]:
theta=coodrdinateDescentRegression(normalize(train_X), train_Y, epoch=100)
[RMSE(train_Y, predict(normalize(train_X),theta)), RMSE(test_Y, predict(normalize(test_X),theta))]

[1.7875163954014147e+78, 1.8246922553272375e+78]

In [154]:
theta=sgdRegression(normalize(train_X), train_Y, alpha = 0.1, epoch=500)
[RMSE(train_Y, predict(normalize(train_X),theta)), RMSE(test_Y, predict(normalize(test_X),theta))]

[9.966905135928979, 9.426698817608205]

In [170]:
lmda=0.01
theta=gradientDescentAutogradLasso(scaler.fit_transform(train_X), train_Y, alpha = 0.1, lmda=0.01, epoch=10)
[RMSE(train_Y, predict(normalize(train_X),theta)), RMSE(test_Y, predict(normalize(test_X),theta))]

[9.662350212063898e+84, 9.628347893516075e+84]